<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/instrGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction Generation

## Basic includes

In [1]:
!pip install ipython-autotime --quiet
%load_ext autotime

time: 0 ns (started: 2022-04-04 22:31:31 +02:00)


## Connect colab and set paths

In [2]:
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive/')
except:
    # local exec
    dataPath = os.path.join(os.getcwd(), '../../../data/2022_03_19')
    logDir = os.path.join(os.getcwd(), 'logs/instruction/')
    saveDir = os.path.join(os.getcwd(), 'model/instruction/')

    sys.path.append(os.getcwd())
else: 
    rootDir = '/content/drive/MyDrive/'

    TIMESTAMP = '2022_03_30'

    dataPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP
    logDir = rootDir + 'Colab Notebooks/recime/LSTM/torch/logs/instruction/'
    saveDir = rootDir + 'Colab Notebooks/recime/LSTM/torch/model/instruction/'

    sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM/torch')

if not os.path.exists(logDir):
  os.makedirs(logDir)
if not os.path.exists(saveDir):
  os.makedirs(saveDir)


time: 16 ms (started: 2022-04-04 22:31:31 +02:00)


## Basic includes

In [3]:
import pandas as pd
import numpy as np

import torch

import importlib

import pickle
import yaml


time: 4.47 s (started: 2022-04-04 22:31:32 +02:00)


In [4]:
import src.data as dataLib
import src.models as modelLib

time: 12.3 s (started: 2022-04-04 22:31:36 +02:00)


## Seed

In [5]:
torch.manual_seed(0)
np.random.seed(0)

time: 16 ms (started: 2022-04-04 22:31:49 +02:00)


## Training execution

### Set device

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print(torch.cuda.get_device_name(0))

time: 0 ns (started: 2022-04-04 22:31:49 +02:00)


### Set hyperparams

In [7]:
importlib.reload(modelLib)

hp = modelLib.HyperParams(epochs=1, batchSize=2, hiddenDim=10, numLayers=1, embeddingDim=100) # just for local test
# hp = modelLib.HyperParams(epochs=30, batchSize=32, hiddenDim=256, numLayers=1, embeddingDim=300)
print(hp)

epochs 1
batchSize 2
lr 0.001
ratio train|val [0.7, 0.3]
hiddenDim 10
numLayers 1
embeddingDim 100

time: 16 ms (started: 2022-04-04 22:31:49 +02:00)


### Set dataset

In [8]:
importlib.reload(dataLib)

instSet = dataLib.InstructionSet(dataPath, setSize=2)

testSet = dataLib.getPreProcData(dataPath, range(-1,0))

d:\02_Studium\SBX\mad-recime\network\LSTM\torch\src\data.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))


time: 4.22 s (started: 2022-04-04 22:31:50 +02:00)


d:\02_Studium\SBX\mad-recime\network\LSTM\torch\src\data.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))


In [9]:
wordCount = pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_counts).sort_values())
wordCount

,0
deg,1
drying,1
shut,1
cavity,1
moisten,1
...,...
a,269
in,303
and,556
the,570


time: 47 ms (started: 2022-04-04 22:31:54 +02:00)


In [10]:
pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_index))

,0
OOV,1
\n,2
the,3
and,4
in,5
...,...
verde,1662
curd,1663
dj's,1664
dishes,1665


time: 15 ms (started: 2022-04-04 22:31:55 +02:00)


### Set model

In [11]:
importlib.reload(modelLib)

model = modelLib.EmbedLSTM(hp, instSet.tokenizer, device)
model.to(device)
print(model)


EmbedLSTM(
  (word_embeddings): Embedding(1666, 100, padding_idx=0)
  (lstm): LSTM(100, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=1666, bias=True)
)
time: 31 ms (started: 2022-04-04 22:31:55 +02:00)


### Train

In [12]:
importlib.reload(modelLib)

modelLib.train(instSet, model, hp, device, logDir)

Epoch: 1, loss:    7.31105, acc:    0.00455
time: 4.56 s (started: 2022-04-04 22:31:56 +02:00)


### Test

In [13]:
importlib.reload(modelLib)

idx = np.random.randint(0, len(testSet))
inputIngs = testSet['ingredient'].values[idx]
inputTitle = testSet['title'].values[idx]
seq = modelLib.testInputPrep(inputIngs + inputTitle.split(), instSet.tokenizer)
instPred = modelLib.sample(model, instSet.tokenizer, 300, device, initial=seq).split(instSet.delimiter)

print('Input ings: {}\nInput Title: {}\nPred. Instructions:\n{}'.format(inputIngs, inputTitle, instPred))

Input ings: ['acorn squash', 'ground beef', 'salt', 'cinnamon', 'apples', 'raisins', 'salt', 'brown sugar', 'margarine']
Input Title: Apple Filled Squash Halves
Pred. Instructions:
['bit individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual individual

## Save model

In [14]:
torch.save(model.state_dict(),
           saveDir + 'instGenerator_model_state_dict.pt')

with open(saveDir + 'instTok.pkl', 'wb') as outp:
    pickle.dump(instSet.tokenizer, outp, -1)

with open(saveDir + 'hp.yml', 'w') as outp:
    yaml.dump(dict(
        epochs=hp.epochs,
        batchSize=hp.batchSize,
        lr=hp.lr,
        ratio=hp.ratio,
        hiddenDim=hp.hiddenDim,
        numLayers=hp.numLayers,
        embeddingDim=hp.embeddingDim,
        delimiter=instSet.delimiter
    ), outp, default_flow_style=False)


time: 31 ms (started: 2022-04-04 22:32:01 +02:00)


# Tensorboard visualization

In [15]:
# print path to logs that needs to be inserted for tensorboard
logDir

'd:\\02_Studium\\SBX\\mad-recime\\network\\LSTM\\torch\\logs/instruction/'

time: 0 ns (started: 2022-04-04 22:32:02 +02:00)


In [16]:
%load_ext tensorboard
%tensorboard --logdir='/content/drive/MyDrive/Colab Notebooks/recime/logs/instruction/'

Reusing TensorBoard on port 6006 (pid 3920), started 0:02:35 ago. (Use '!kill 3920' to kill it.)

time: 31 ms (started: 2022-04-04 22:32:02 +02:00)
